<a href="https://colab.research.google.com/github/juanpajedrez/pytorch_learning/blob/main/05_pytorch_going_modular_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:**

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [1]:
# Let's create a src folder
import os

if os.path.exists(os.path.join(os.getcwd(), "src")) == False:
  os.makedirs("src")

In [2]:
# YOUR CODE HERE
%%writefile src/get_data.py
'''
Contains functionality to download the pizza, steak, sushi dataset
'''

import os
import zipfile
from pathlib import Path
import requests

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it.
if image_path.is_dir():
  print(f"{image_path} directory exists")
else:
  print(f"Did not find {image_path}, creating one")
  image_path.mkdir(parents=True, exist_ok = True)

# Donwload pizz, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print("Downloading pizza, steak, and sushi data")
  f.write(request.content)

#Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
  print("Unzipping pizza, steak sushi data")
  zip_ref.extractall(image_path)

#Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

Writing src/get_data.py


In [3]:
# Example running of get_data.py
!python src/get_data.py

Did not find data/pizza_steak_sushi, creating one
Unzipping pizza, steak sushi data


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular).

### Testing & Learning: create `data_setup.py` with a function called create_dataloaders.

In [4]:
%%writefile src/data_setup.py
'''
Contains functionality for creating PyTorch DataLoaders for
image classification data,
'''

import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()
def create_dataloaders(
    train_dir:str,
    test_dir:str,
    transform:transforms.Compose,
    batch_size:int,
    num_workers:int = NUM_WORKERS
):
  '''
  Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns them into
  PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple (train_dataloader, test_dataloader, class_names).
    Where class_names is a lit of the target classes.

  Example usage:
    train_dataloader, test_dataloader, class_names =
      create_dataloaders(train_dir=path/to/train_dir,
      test_dir=path/to/test_dir,
      transform=some_transform,
      batch_size=32,
      num_workers=4)
  '''
  # Create a simple transform
  data_transform = transforms.Compose([
      transforms.Resize((64, 64)),
      transforms.ToTensor(),
  ])

  # Use ImageFolder to create dataset(s)
  train_data = datasets.ImageFolder(root=train_dir,
                                    transform=data_transform,
                                    target_transform=None)

  test_data = datasets.ImageFolder(root=test_dir,
                                  transform=data_transform)

  # Get the class names
  class_names = train_data.classes

  # Create train and test dataloaders
  train_dataloader = DataLoader(
      dataset=train_data,
      batch_size = batch_size,
      shuffle = True,
      num_workers = num_workers,
      pin_memory = True
  )

  test_dataloader = DataLoader(
    dataset=test_data,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=False,
    pin_memory=True)

  return train_dataloader, test_dataloader, class_names

Writing src/data_setup.py


### Testing and learning: Creating a model (TinyVGG) with a script (`model_builder.py`).

In [5]:
%%writefile src/model_builder.py
'''
Contains PyTorch model to instantiage a TinyVGG model from
CNN Explainer website:
https://poloclub.github.io/cnn-explainer/
'''
import torch
from torch import nn

class TinyVGG(nn.Module):
  '''
  Creates the TinyVGG Architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  '''
  def __init__(self, input_shape:int, hidden_units:int, output_shape:int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3, # how big is the square that's going over the image?
                  stride=1, # default
                  padding=0), # options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                      stride=2) # default stride value is same as kernel_size
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        # Where did this in_features shape come from?
        # It's because each layer of our network compresses and changes the shape of our inputs data.
        nn.Linear(in_features=hidden_units*13*13,
                  out_features=output_shape)
    )

  def forward(self, x: torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x

Writing src/model_builder.py


### Testing and learning: create `engine.py` using `train_step`, `test_step`, and `train` cells.

In [6]:
%%writefile src/engine.py
'''
This trains a PyTorch image classification model using device-agnostic
code.
'''
import torch
from typing import Tuple, Dict, List
from tqdm import tqdm

def train_step(model:torch.nn.Module,
               dataloader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer:torch.optim.Optimizer,
               device:torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)

  """
  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
    # Send data to target device
    X, y = X.to(device), y.to(device)

    # 1. Forward pass
    y_pred = model(X)

    # 2. Calculate the accumulate loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step, step, step
    optimizer.step()

    # Calculate and accumulate accuracy metric across all batches
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim = 1), dim = 1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
  """
  # Put the model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on the inference context manager
  with torch.inference_mode():
    # Loop through DataLoader batches
    for batch, (X, y) in enumerate(dataloader):
      # Send the data to target device
      X, y = X.to(device), y.to(device)

      #1. Forward pass
      test_pred_logits = model(X)

      #2. Calculate and accumulate the loss
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()

      # Calculate and accumulate accuracy
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List[float]]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  # Create empty results dictionary
  results = {"train_loss": [],
             "train_acc": [],
             "test_loss": [],
             "test_acc": []
  }

  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
      train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
      test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

      # Print out what's happening
      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

      # Update results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results

Writing src/engine.py


### Testing and Learning: Create a function to save model

In [7]:
%%writefile src/utils.py
'''
Fle containing various utility functions for PyTorch model training.
'''
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing src/utils.py


In [13]:
%%writefile src/train.py
'''
This trains a PyTorch image classification model using device-agnostic
code.
'''
# Import necessary modules
import argparse
from pathlib import Path
import torch
import os
from torchvision import transforms
import data_setup, engine, model_builder, utils
from timeit import default_timer as timer

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# Have some default directories
train_dir = image_path / "train"
test_dir = image_path / "test"

# Create argparsers and add arguments
parser = argparse.ArgumentParser()
parser.add_argument("-trd",
                    "--train_dir",
                    type = str,
                    default = train_dir,
                    help = "String path: Training Directory")
parser.add_argument("-ted",
                     "--test_dir",
                     type = str,
                     default = test_dir,
                     help = "String path: Testing Directory")
parser.add_argument("-lr",
                      "--learning_rate",
                      type = float,
                      default = 0.001,
                      help = "Float: Training learning rate")
parser.add_argument("-b",
                      "--batch_size",
                      type = int,
                      default = 16,
                      help = "Integer: Batch size")
parser.add_argument("-ne",
                     "--num_epochs",
                     type = int,
                     default = 5,
                     help = "Integer: Number of epochs")
parser.add_argument("-hu",
                     "--hidden_units",
                     type = int,
                     default = 10,
                     help = "Integer: Number of hidden units TinyVGG")

#Obtain the arguments
args = parser.parse_args()

#Setup hyperparameters with parser
TRAIN_DIR = args.train_dir
TEST_DIR = args.test_dir
LEARNING_RATE = args.learning_rate
BATCH_SIZE = args.batch_size
NUM_EPOCHS = args.num_epochs
HIDDEN_UNITS = args.hidden_units

#Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

#Create transforms
data_transform = transforms.Compose(
    [
        transforms.Resize((64, 64)),
        transforms.ToTensor()
    ]
)

#Create Dataloader's and get class_names
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=TRAIN_DIR,
    test_dir=TEST_DIR,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

#Create the model
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units = HIDDEN_UNITS,
    output_shape = len(class_names)).to(device)

#Setup loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                              LEARNING_RATE)

# Start the timer
start_time = timer()

#Train the model
engine.train(
  model=model,
  train_dataloader=train_dataloader,
  test_dataloader=test_dataloader,
  loss_fn=loss_fn,
  optimizer=optimizer,
  epochs=NUM_EPOCHS,
  device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

#Save the model using utils
utils.save_model(
  model = model,
  target_dir = "models",
  model_name = "saved_tinyvgg_model.pth"
)

Overwriting src/train.py


In [14]:
# Example running of train.py
!python src/train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003 --num_epochs 15

  0% 0/15 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1015 | train_acc: 0.2915 | test_loss: 1.1005 | test_acc: 0.3467
  7% 1/15 [00:01<00:24,  1.74s/it]Epoch: 2 | train_loss: 1.0899 | train_acc: 0.3537 | test_loss: 1.0833 | test_acc: 0.3467
 13% 2/15 [00:02<00:17,  1.35s/it]Epoch: 3 | train_loss: 1.0753 | train_acc: 0.3910 | test_loss: 1.0676 | test_acc: 0.4267
 20% 3/15 [00:03<00:14,  1.24s/it]Epoch: 4 | train_loss: 1.0687 | train_acc: 0.4657 | test_loss: 1.0574 | test_acc: 0.4133
 27% 4/15 [00:04<00:12,  1.17s/it]Epoch: 5 | train_loss: 1.0341 | train_acc: 0.4720 | test_loss: 1.0474 | test_acc: 0.4267
 33% 5/15 [00:06<00:11,  1.12s/it]Epoch: 6 | train_loss: 0.9991 | train_acc: 0.5240 | test_loss: 1.0132 | test_acc: 0.4400
 40% 6/15 [00:07<00:09,  1.10s/it]Epoch: 7 | train_loss: 0.9415 | train_acc: 0.5946 | test_loss: 0.9991 | test_acc: 0.4800
 47% 7/15 [00:08<00:09,  1.14s/it]Epoch: 8 | train_loss: 0.8857 | train_acc: 0.6232 | test_loss: 0.9840 | test_acc: 0.4133
 53% 8/15 [00:09<00:08, 

## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function).
* You may also have to write code to load in a trained model.

In [54]:
# YOUR CODE HERE
%%writefile src/predict.py

# Import modules to read it
import argparse
import torch
from torchvision import transforms, io
import model_builder
import matplotlib.pyplot as plt
# Let's add a parser to access an image
parser = argparse.ArgumentParser()

# Lets add an argument
parser.add_argument("-i",
                    "--image",
                    type = str,
                    default = "data/pizza_steak_sushi/test/sushi/175783.jpg",
                    help = "String path to image")

# Let's obtain in
args = parser.parse_args()

# Setup the device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Read the custom image path
custom_image_path = args.image

# Load in the custom image and convert to torch.float32
custom_image = io.read_image(str(custom_image_path)).type(torch.float32) / 255.

# Transform the image
#Create transforms
data_transform = transforms.Compose(
    [
        transforms.Resize((64, 64)),
    ]
)

custom_image_transformed = data_transform(custom_image)

#Create the model
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units = 128,
    output_shape = 3 ).to(device)

    # UHMMM, many here were hardcoded :(, we need to save the past args somehowlater in future

# Load the saved model
model.load_state_dict(torch.load(f = "models/saved_tinyvgg_model.pth"))
model.eval()
with torch.inference_mode():
  custom_image_pred = model(custom_image_transformed.to(device).unsqueeze(dim = 0))

plt.figure()
print(custom_image_transformed.shape)
plt.imshow(custom_image_transformed.permute(1, 2, 0))
plt.show()

# Convert logits -> prediction probabilities -> prediction labels
print(torch.softmax(custom_image_pred, dim = 1).argmax(dim = 1))

Overwriting src/predict.py


In [55]:
# Example running of predict.py
!python src/predict.py --image data/pizza_steak_sushi/test/sushi/175783.jpg

torch.Size([3, 64, 64])
Figure(640x480)
tensor([2], device='cuda:0')
